In [9]:
#Reset Button. Press when memory goes crazy fat !
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
# imports cell
import numpy as np
import _pickle as cPickle
import sys 
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Deep Learning libs
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Conv2D,ZeroPadding2D
from keras.callbacks import EarlyStopping,ModelCheckpoint

Using TensorFlow backend.


# Let's start with data

We use DeepSig Dataset: RadioML 2016.04C<br>
A synthetic dataset, generated with GNU Radio, consisting of 11 modulations. This is a
variable-SNR dataset with moderate LO drift, light fading, and numerous different
labeled SNR increments for use in measuring performance across different signal and
noise power scenarios.

## Loading data

**The file is formatted as a "pickle" file which can be open for example in python by using cPickle.load(...)**



In [2]:
filename = "RML2016.10b.dat"
open_file = open(filename,'rb')
data = cPickle.load(open_file, encoding='latin1')

In [3]:
# Use lists when accessing data from dict for ease of access.
keys_list = list(data.keys())
temp_list = []
label_list = []
snr_list = []
for i in range(len(keys_list)):
    curr_item = data[keys_list[i]] 
    temp_list.append(curr_item)
    for j in range(curr_item.shape[0]):
        label_list.append(keys_list[i][0])
    #for j in rane(curr_item.shape[0]):
    #    snr_list.append(keys_list[i][1])

# Convert all lists into numpy arrays.
X = np.array(temp_list).reshape(1200000,2,128)
Y = np.array(label_list)

# Clear All un-neccsarry lists created.
temp_list.clear()
label_list.clear()
snr_list.clear()
keys_list.clear()
data.clear()

## Create feature Spaces for data

**Every sample is presented using two vectors each of them has 128 elements. Those two vectors are the in-phase and quadrature phase components of a sample.**

In [4]:
derivative_X = np.arange(1200000*4*128, dtype="float32").reshape(1200000,4,128)
derivative_X = np.concatenate((np.apply_along_axis(lambda column:np.gradient(column),2,X),X),axis=1)
del X

In [5]:
integral_X = np.arange(1200000*4*128, dtype="float32").reshape(1200000,4,128)
integral_X =np.concatenate((np.apply_along_axis(lambda column:np.hstack((0,column)),2,np.apply_along_axis(lambda column:sp.integrate.cumtrapz(column,dx=1,axis=-1),2,X)),X),axis=1)

## Splitting data

**Split the data into 50% for training/validation and 50% for testing.**

In [5]:
#(trainX, testX, trainY, testY) = train_test_split(X,Y, test_size=0.50, random_state=42)
(trainX, testX, trainY, testY) = train_test_split(derivative_X,Y, test_size=0.50, random_state=42)

In [6]:
del derivative_X

## Encoding Labels

Our class labels are currently represented as strings; however, Keras will assume that both:

1. Labels are encoded as integers

2. And furthermore, one-hot encoding is performed on these labels making each label represented as a vector rather than an integer

To accomplish this encoding, we can use the LabelBinarizer  class from scikit-learn

In [7]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# Defining the Fully Connected Model
<ul>

1. The core data structure of Keras is a model. The simplest type of model is the Sequential model, a linear stack of layers.

2. A 3-layer deep neural network consisiting only of fully connected layers of size 512, 256, 11 neurons.


In [8]:
# Declare layers size
in_shape = (4,128)
hidden1_size = 512
hidden2_size = 256
hidden3_size = 10

# Build model
model = Sequential()
model.add(Dense(hidden1_size, input_shape = in_shape))
model.add(Activation('relu'))
model.add(Dense(hidden2_size))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(hidden3_size))
model.add(Activation('softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4, 512)            66048     
_________________________________________________________________
activation_1 (Activation)    (None, 4, 512)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 4, 256)            131328    
_________________________________________________________________
activation_2 (Activation)    (None, 4, 256)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                10250     
_________________________________________________________________
acti

## Fitting the model

1. The training process will run for a fixed number of iterations (epochs) through the dataset.

2. We can also set the number of instances that are evaluated before a weight update in the network is performed (batch size).


In [9]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model_2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(trainX, trainY, epochs=150, batch_size=100000, validation_split=0.05, callbacks=[es,mc])

Instructions for updating:
Use tf.cast instead.
Train on 570000 samples, validate on 30000 samples
Epoch 1/150
570000/570000 [==============================] - 36s 63us/step - loss: 2.3007 - acc: 0.1443 - val_loss: 2.2971 - val_acc: 0.1626

Epoch 00001: val_acc improved from -inf to 0.16263, saving model to best_model_2.h5
Epoch 2/150
570000/570000 [==============================] - 34s 60us/step - loss: 2.2929 - acc: 0.1637 - val_loss: 2.2828 - val_acc: 0.1697

Epoch 00002: val_acc improved from 0.16263 to 0.16970, saving model to best_model_2.h5
Epoch 3/150
570000/570000 [==============================] - 35s 61us/step - loss: 2.2727 - acc: 0.1668 - val_loss: 2.2492 - val_acc: 0.1707

Epoch 00003: val_acc improved from 0.16970 to 0.17070, saving model to best_model_2.h5
Epoch 4/150
570000/570000 [==============================] - 33s 59us/step - loss: 2.2309 - acc: 0.1721 - val_loss: 2.1917 - val_acc: 0.1794

Epoch 00004: val_acc improved from 0.17070 to 0.17937, saving model to best

570000/570000 [==============================] - 31s 55us/step - loss: 1.5932 - acc: 0.3745 - val_loss: 1.5979 - val_acc: 0.3735

Epoch 00073: val_acc improved from 0.37287 to 0.37350, saving model to best_model_2.h5
Epoch 74/150
570000/570000 [==============================] - 32s 57us/step - loss: 1.5893 - acc: 0.3760 - val_loss: 1.5949 - val_acc: 0.3743

Epoch 00074: val_acc improved from 0.37350 to 0.37427, saving model to best_model_2.h5
Epoch 75/150
570000/570000 [==============================] - 31s 54us/step - loss: 1.5860 - acc: 0.3769 - val_loss: 1.5922 - val_acc: 0.3721

Epoch 00075: val_acc did not improve from 0.37427
Epoch 76/150
570000/570000 [==============================] - 30s 53us/step - loss: 1.5833 - acc: 0.3761 - val_loss: 1.5901 - val_acc: 0.3753

Epoch 00076: val_acc improved from 0.37427 to 0.37527, saving model to best_model_2.h5
Epoch 77/150
570000/570000 [==============================] - 34s 60us/step - loss: 1.5802 - acc: 0.3784 - val_loss: 1.5868 - val_

# Defining the Convolutional Neural Network

In [ ]:
# Declare layers size
in_shape = (128,2,1)
conv1_kernel_shape=(3,1)
conv1_number_of_filters=64
conv2_kernel_shape=(3,2)
conv2_number_of_filters=16
dense1_size = 128
dense2_size = 10


# Build model
model_conv = Sequential()
model_conv.add(Reshape((None,128,2,1), input_shape=in_shape))
model_conv.add(Conv2D(conv1_number_of_filters, conv1_kernel_shape, strides=1,
                 padding='same', data_format='channels_last',activation='relu'))
model_conv.add(Conv2D(conv2_number_of_filters, conv2_kernel_shape, strides=1,
                 padding='same', data_format='channels_last',activation='relu'))
model_conv.add(Flatten())
model_conv.add(Dense(dense1_size, activation='relu'))
model_conv.add(Dense(dense2_size, activation='softmax'))

# Compile model
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_conv.summary()